In [89]:
from math import factorial, floor, ceil

def print_table(arr, rows=[], columns=[]):
    if len(rows) > 0:
        for i in range(len(rows)):
            arr[i] = [str(rows[i])] + arr[i]

    if len(columns) > 0:
        arr = [[' '] + list(map(str, columns))] + arr

    row_lengths = [max(map(lambda x: len(str(x)), column)) for column in zip(*arr)]

    for row in arr:
        for i in range(len(row)):
            padding = row_lengths[i] - len(str(row[i])) + 2
            print(f'{" " * floor(padding / 2)}{row[i]}{" " * ceil(padding / 2)}', end='')
        print()

def calculate(lamb, mu, n, assistance=False):
    ro = lamb / mu
    p0 = 1 / sum((1 / factorial(k)) * ro ** k for k in range(n))

    p_cancelation = (1 / factorial(n) * (lamb / (n * mu))**n) * (p0 ** n if assistance else p0)
    # p_cancelation = 1 / (1 + sum([(ro ** k) / factorial(k) for k in range(n)]))
    Q = 1 - p_cancelation
    A = lamb * Q
    n_o = A / mu

    # print(f'Вероятность отказа заявки: {p_cancelation}')
    # print(f'Относительная пропускная способность СМО: {Q}')
    # print(f'Абсолютную пропускную способность: {A}')
    # print(f'Среднее число занятых обслуживанием каналов: {n_o}')

    return [p_cancelation, Q, A, n_o]
    
n = 3
lamb = 4
mu = 0.5

no_assistance = calculate(lamb, mu, n)
unlimited_assistance = calculate(lamb, mu, n, assistance=True)
uniform_assistance = calculate(lamb, mu, n, assistance=True)

print_table([no_assistance, unlimited_assistance, uniform_assistance], 
            rows=['Без взаимопомощи', 'Неограниченная', 'Равномерная'], 
            columns=['Вероятность отказа', 'Относительная пропускная способность', 'Абсолютая пропускная способность', 'Среднее число занятых каналов'])

                   Вероятность отказа   Относительная пропускная способность  Абсолютая пропускная способность  Среднее число занятых каналов 
 Без взаимопомощи  0.07708521529659738           0.9229147847034026                  3.6916591388136104               7.383318277627221       
  Неограниченная   4.5856761033074e-05           0.9999541432389669                  3.9998165729558677               7.999633145911735       
   Равномерная     4.5856761033074e-05           0.9999541432389669                  3.9998165729558677               7.999633145911735       
